## Extract the data from the RINEX files. ##

This notebook creates two files a csv file and JuliaDB table  from the information on the RINEX file that contains the ionospheric Total Electron Content data.

In [1]:
using CSV
using Dates
using IJulia

In [2]:
ionex_directory = "../ionex_esag/"
tec_csv_directory = "..tec_csv_esag/"
tec_table_directory = "../tec_tables_esag/"


"../tec_tables_esag/"

**Read the contents of the RINEX file**

In [5]:
# OPen the file and place its contents in an array name lines
function read_file(ionex_file)
    #print("Reading", ionex_file, "\n")
    file = open(ionex_file);
    lines = readlines(file)
    close(file)
    return lines  
end

read_file (generic function with 1 method)

In [6]:
function get_map_values(lines)
    # This function could be improved by reading only the TEC map portion of the file
    #print("Inside get_map_values \n")
    rms_map = 0  # Some files do not have a complete rms map 
    index = 0
    tec_values = []   # Tec values per longitud, lat(-180,180)
    tec = []          # All the tec values in a file
    dates_array = []  # Store the different time stamps
    
    for line in lines
        index += 1
        
        # Obtain time_stamp and tec values for every map (13 TEC maps in a file)
        if occursin(r"START OF TEC MAP", line)
            date_line = lines[index+1]
            date_units = []
            string = split(date_line)
            
            for x in 1:6
                push!(date_units,parse(Int,string[x]))
            end
            
            # Date units 
            year = date_units[1]
            month = date_units[2]
            day = date_units[3]
            hour = date_units[4]
            minutes = date_units[5]
            seconds = date_units[6]
            
            # Map time_stamp for a period(2 hours)
            time_stamp = DateTime(year, month, day, hour, minutes, seconds)
            push!(dates_array, time_stamp)
        end
        
        if occursin(r"START OF RMS MAP", line)
            rms_map = 1
            #print("RMS MAP reached!\n")
            return dates_array, tec
        end
        
        # Tec values per latitud in the longitud range (-180,180)
        if occursin(r"LAT/LON1/LON2/DLON/H", line) && (rms_map == 0)
            for x in 1:5
                tec_values = split(lines[index + x])
                for n in tec_values
                    #if occursin(r"^-?[0-9]*$", n)
                    if occursin(r"^-?\d*\.{0,1}\d+$", n)
                        push!(tec, n)
                    else
                        push!(tec, 0)
                        print("Issue found in ", dates_array[1], n, "\n")
                    end
                end
            end
        end
        
    end
    return dates_array, tec
end

get_map_values (generic function with 1 method)

In [7]:
# used to create a latitud, to hold 2-hour-interval Tec map data
function create_latitud(start, stop, step)
#    print("Inside create_latitud function \n")
    coordinates = []
    for i in range(start, stop=stop, step=step)
        for j in 1:73
            push!(coordinates,i)
        end
    end
    coordinates # Julia automatically returns the last statement
end
    

create_latitud (generic function with 1 method)

In [8]:
function get_coordinates()
#    print("Inside get_coordinates\n")
    latitud = create_latitud(87.5, -87.5, -2.5)
    longitud = collect(Iterators.flatten([collect(-180:5:180) for i in 1:71])) # I Still don't grok this!
    return latitud, longitud
end

get_coordinates (generic function with 1 method)

In [9]:
# Create CSV for daily tec map data.
function create_csv(directory, ionex_file)
   # print("Inside create_csv\n")
    lines = read_file(string(directory, ionex_file))
    dates_array, tec = get_map_values(lines)
    latitud, longitud = get_coordinates()
    
    csv_file = string(tec_csv_directory, ionex_file, ".csv")
    table_file = string(tec_table_directory, ionex_file, "_table")
    
    start = 1
    map_tec_values = 5183  # Total tec values in a 2 hour map period
    stop = map_tec_values
    #print("Starting to arrange data for CSV\n")
    for d in dates_array
        #print("\nStart ", start)
        time_stamp = [d for i in 1:map_tec_values]
        #print("Looking at TEC value\n")
        tec_value = [parse(Int,value) for value in tec[start:stop]]
        t = table((time_stamp=time_stamp, latitud=latitud, longitud=longitud, tec_value=tec_value); pkey = [:time_stamp, :latitud, :longitud, :tec_value])
        
        CSV.write(csv_file ,t, append=true)
        JuliaDB.save(t, table_file)
        
        start += map_tec_values
        stop += map_tec_values
    end
end


create_csv (generic function with 1 method)

In [10]:
function create_database()
    directory = ionex_directory
    for f in files
        if isfile(string(tec_csv_directory, f, ".csv")) == false
            print("Writting file:", f, "\n")
            create_csv(directory, f)
        end
    end
end

create_database (generic function with 1 method)

In [11]:
# Read the contents of the directory in order to process the files to create the csv 
files = readdir(ionex_directory)
@time create_database()

  0.018021 seconds (28.71 k allocations: 2.667 MiB)
